Machine vision model to classify defects in sewer systems

In [1]:
!pip install split_folders
import tensorflow as tf
import sklearn
import splitfolders
import os
from tensorflow.keras import datasets, layers, models
import numpy as np

goes through copied dataset and removes gdot files

In [2]:
input_folder = "/content/drive/MyDrive/machine vision/copieddata" #Enter Input Folder

In [4]:
#goes through folder to check for anything not jpg, jpeg or png to remove
for class_dir in os.listdir(input_folder):
    class_path = os.path.join(input_folder, class_dir)
    if not os.path.isdir(class_path):
        continue

    for filename in os.listdir(class_path):
        if filename.startswith('.') or not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_path = os.path.join(class_path, filename)
            print(f"Removing: {file_path}")
            os.remove(file_path)

copies dataset into new folder splitting the data into 80% training 20% test

In [ ]:
output = "/content/drive/MyDrive/machine vision/splitdata" #Enter Output Folder

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(0.8,0.2))

Copying files: 27267 files [46:35,  9.75 files/s]


source: https://stackoverflow.com/questions/57394135/split-image-dataset-into-train-test-datasets

creates paths so that parts of dataset can be refrenced

In [3]:
dataset_path   = '/content/drive/MyDrive/machine vision/splitdata/'
train_path = '/content/drive/MyDrive/machine vision/splitdata/train' #creates paths for training and testing
validity_path = '/content/drive/MyDrive/machine vision/splitdata/val'

In [4]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=(224, 224),  # Resizes pictures
    batch_size=32
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    validity_path,
    image_size=(224, 224),
    batch_size=32
)

class_names = train_dataset.class_names
print(class_names)  #creates class names from folders

Found 21935 files belonging to 75 classes.
Found 5488 files belonging to 75 classes.
['B', 'CC', 'CL', 'CM', 'CN', 'CS', 'CU', 'CXD', 'CXI', 'D', 'DB', 'DEC', 'DEE', 'DEF', 'DEG_1', 'DEG_2', 'DER', 'DES', 'EX', 'FC', 'FL', 'FM', 'FS', 'H', 'ID', 'IG', 'ING', 'INGF', 'INGZ', 'IR', 'IS', 'JDL', 'JDM_1', 'JDM_2', 'JN', 'JX', 'JXB', 'JXD', 'LD', 'LL', 'LR', 'LU', 'LX', 'MB', 'MC', 'MM', 'OB', 'OBB', 'OBC', 'OBI', 'OBM', 'OBP', 'OBS', 'OBX', 'OJL', 'OJM', 'RF', 'RM_1', 'RM_2', 'RT', 'SAP', 'SAV', 'SCP', 'SR', 'SRC', 'SRP', 'SRV', 'SS', 'SV', 'SW', 'VR', 'VV', 'WL', 'XB', 'XP']


In [5]:
num_classes = 75

base_model = tf.keras.applications.EfficientNetB0( #creates model using efficientdet
    include_top=False,
    input_shape=(224, 224, 3),
    pooling='avg',
    weights='imagenet'
)

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    base_model,
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes)
])

In [6]:
model.compile(optimizer='adam', #compiles model
       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
       metrics=['accuracy'])

history = model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
 26/686 ━━━━━━━━━━━━━━━━━━━━ 2:03:50 11s/step - accuracy: 0.0632 - loss: 4.0781

KeyboardInterrupt: 

Model seemed to work however compiling would take so long I did not wait to see the accuracy after compiling